In [3]:
%pip install flask
%pip install pdfplumber
%pip install ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import requests
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
from PyPDF2 import PdfReader
from docx import Document

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = './uploads'
app.config['ALLOWED_EXTENSIONS'] = {'pdf', 'docx'}
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB

# Ensure upload directory exists
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

def extract_text(filepath, extension):
    text = ""
    try:
        if extension == 'pdf':
            with open(filepath, 'rb') as f:
                reader = PdfReader(f)
                for page in reader.pages:
                    text += page.extract_text()
        elif extension == 'docx':
            doc = Document(filepath)
            for para in doc.paragraphs:
                text += para.text + '\n'
        return text.strip()
    except Exception as e:
        print(f"Error extracting text: {e}")
        return None

def analyze_resume(text):
    prompt = f"""Analyze this resume against software engineering job criteria:
    - Check for relevant technical skills
    - Evaluate work experience duration and relevance
    - Look for educational qualifications
    - Identify key projects and achievements
    - Assess any certifications
    
    Resume Content:
    {text}
    
    Provide a detailed analysis in this JSON format:
    {{
        "fit_score": 0-100,
        "strengths": [],
        "weaknesses": [],
        "missing_skills": [],
        "recommendations": []
    }}"""
    
    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                'model': 'deepseek-r1',
                'prompt': prompt,
                'stream': False,
                'format': 'json'
            }
        )
        response.raise_for_status()
        return response.json()['response']
    except Exception as e:
        print(f"Error analyzing resume: {e}")
        return None

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        # Process the file
        extension = filename.rsplit('.', 1)[1].lower()
        resume_text = extract_text(filepath, extension)
        
        if not resume_text:
            return jsonify({'error': 'Failed to extract text from file'}), 500
        
        analysis = analyze_resume(resume_text)
        if not analysis:
            return jsonify({'error': 'Failed to analyze resume'}), 500
        
        # Clean up uploaded file
        try:
            os.remove(filepath)
        except:
            pass
        
        return jsonify({
            'message': 'Analysis complete',
            'result': analysis
        })
    
    return jsonify({'error': 'Invalid file type'}), 400

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5080
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/vikk/Documents/GitHub/Innoverse2.0Submission/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/vikk/Documents/GitHub/Innoverse2.0Submission/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/vikk/Documents/GitHub/Innoverse2.0Submission/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vikk/Documents/GitHub/Innoverse2.0Submission/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/vikk/Documents/GitH

SystemExit: 1